# Data Preparation

This notebook cleans and prepares the data for training, validation, and testing.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path
import pickle
import glob
import PIL
import copy

### Filter out filler pages

Remove filler pages that do not contain sheet music information (e.g. title, foreword).  The file filler.txt indicates which pages are filler.

In [ ]:
def getFillerList(filler_file, feat_dir):
    d = {} # list of pages to remove
    with open(filler_file, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) > 1:
                relpath = parts[0] # e.g. Bach/00748
                scoreID = os.path.basename(relpath) # e.g. 00748
                removeField = parts[1].strip('"') # e.g. "0,1,-2,-1" or "r" or "rl"
                numPages = getNumPages(relpath, feat_dir)
                if removeField == 'r' or removeField == 'rl': # remove all pages
                    for pkl_file in glob.glob('{}/{}/*.pkl'.format(feat_dir, parts[0])):
                        pageID = os.path.splitext(os.path.basename(pkl_file))[0] # e.g. 00822-3
                        d[pageID] = 1
                else:
                    for pageNumStr in removeField.split(','):
                        pageNum = int(pageNumStr)
                        if pageNum < 0:
                            pageID = '{}-{}'.format(scoreID, numPages + pageNum)
                            d[pageID] = 1
                        else:
                            pageID = '{}-{}'.format(scoreID, pageNum)
                            d[pageID] = 1
    return d

In [ ]:
def getNumPages(relpath, indir):
    numPages = len(glob.glob('{}/{}/*.pkl'.format(indir, relpath)))
    return numPages

In [ ]:
def getNonFillerFeatures(filler_file, feat_dir):
    '''
    Collect bootleg score features from all pages that are (a) not filler and (b) have a valid 
    bootleg score matrix.
    '''
    
    filler = getFillerList(filler_file, feat_dir)
    feats = {}
    
    for pieceDir in glob.glob('{}/*/*/'.format(feat_dir)): # e.g. score_feat/Bach/00748/
        
        pieceID = pieceDir.split('/')[-2]
        composer = pieceDir.split('/')[-3]
        accum = [] # features from all pages in this score (or None if filler/no features extracted)
        numPages = len(glob.glob('{}/*.pkl'.format(pieceDir)))
        
        for i in range(numPages):
            
            pkl_file = '{}/{}-{}.pkl'.format(pieceDir, pieceID, i)
            pageID = '{}-{}'.format(pieceID, i) # e.g. 00748-2
            if pageID in filler: # filler page, skip
                accum.append(None)
                continue
            with open(pkl_file, 'rb') as f:
                bscore = pickle.load(f)['bscore']
            if bscore is None: # if None, no features were computed
                accum.append(None)
            else:
                accum.append(bscore == 1) # convert from float to bool to compress memory
        
        if len(accum) > 0:
            feats[pieceDir] = accum
    
    return feats

In [ ]:
filler_file = '../cfg_files/filler.txt'
score_feat_dir = '../score_feat'
feats = getNonFillerFeatures(filler_file, score_feat_dir) # key: pieceDir, value: list of bscore matrices

### Investigate Feature Statistics

In [ ]:
def getFeatureStats(feats):
    
    # count number of features per page
    featsPerPage = []
    for pieceDir in feats:
        for elem in feats[pieceDir]:
            if elem is not None:
                featsPerPage.append(elem.shape[1])
    featsPerPage = np.array(featsPerPage)
    printStats(featsPerPage, "Number of Features Per Page")
    
    # plot histogram
    plt.subplot(2,1,1)
    plt.hist(featsPerPage, bins=100)
    plt.xlabel('Number of Events In Single Page')
    plt.ylabel('Frequency')
    plt.show()
    
    # count total number of pages by composer
    pages = {}
    for pieceDir in feats: # e.g. score_feat/Bach/00748/
        composer = pieceDir.split('/')[-3]
        if composer not in pages:
            pages[composer] = 0
        pages[composer] += len([1 for elem in feats[pieceDir] if elem is not None])
    pageCnts = [pages[composer] for composer in pages]
    composers = [composer[0:5] for composer in pages]
    printStats(pageCnts, "Total Number of Pages by Composer")
    
    # plot histogram
    x_pos = np.arange(len(pageCnts))
    plt.bar(x_pos, pageCnts)
    plt.xticks(x_pos, composers)
    plt.ylabel('Total # Pages')
    plt.show()
    
    # count total number of note events by composer
    noteEvents = {}
    for pieceDir in feats: # e.g. score_feat/Bach/00748/
        composer = pieceDir.split('/')[-3]
        if composer not in noteEvents:
            noteEvents[composer] = 0
        for elem in feats[pieceDir]:
            if elem is not None:
                noteEvents[composer] += elem.shape[1]
    noteEventCnts = [noteEvents[composer] for composer in noteEvents]
    printStats(noteEventCnts, "Total Number of Note Events by Composer")
    
    # plot histogram
    x_pos = np.arange(len(composers))
    plt.bar(x_pos, noteEventCnts)
    plt.xticks(x_pos, composers)
    plt.ylabel('Total # Note Events')
    plt.show()
    
    return

In [ ]:
def printStats(arr, title = None):
    if title:
        print(title)
    print('Mean: {}'.format(np.mean(arr)))
    print('Std: {}'.format(np.std(arr)))
    print('Min: {}'.format(np.min(arr)))
    print('Max: {}'.format(np.max(arr)))

In [ ]:
getFeatureStats(feats)

Note that there is a wide variation in the number of features per page and a significant class imbalance in the data.  We will define a proxy task which tries to classify short fixed-length chunks of bootleg score features, and resample the classes to ensure class balance.

### Split Data into Train, Validation, & Test

Below we separate the data into train, validation, & test sets.  We split the data by piece (as opposed to page) to ensure total separation.  This data corresponds to the original task of classifying a single page of sheet music.

In [ ]:
def splitTrainValidTest(d, train=.6, validation=.2, test=.2, savefile = None):
    
    # shuffle
    assert(train + validation + test == 1.0)
    np.random.seed(0)
    pieceDirs = list(d.keys())
    np.random.shuffle(pieceDirs)
    
    # split
    breakpt1 = int(len(pieceDirs) * train)
    breakpt2 = breakpt1 + int(len(pieceDirs) * validation)
    pieceDirs_train = pieceDirs[0:breakpt1]
    pieceDirs_valid = pieceDirs[breakpt1:breakpt2]
    pieceDirs_test = pieceDirs[breakpt2:]
    
    # save
    d_train = getDataSubset(d, pieceDirs_train)
    d_valid = getDataSubset(d, pieceDirs_valid)
    d_test = getDataSubset(d, pieceDirs_test)
    if savefile:
        saveToPickle([d, pieceDirs_train, pieceDirs_valid, pieceDirs_test], savefile)
    
    return d_train, d_valid, d_test

In [ ]:
def getDataSubset(dAll, toKeep):
    dSubset = {}
    for pieceDir in toKeep:
        print(dAll[pieceDir][0])
        dSubset[pieceDir] = dAll[pieceDir]
    return dSubset

In [ ]:
def saveToPickle(d, outfile):
    with open(outfile, 'wb') as f:
        pickle.dump(d, f)

In [ ]:
def loadPickle(infile):
    with open(infile, 'rb') as f:
        d = pickle.load(f)
    return d

In [ ]:
save_pages_file = '{}/data.pages.pkl'.format(score_feat_dir)
d_train, d_valid, d_test = splitTrainValidTest(feats, train=.6, validation=.2, test=.2, savefile=save_pages_file)

### Format data in chunks

Below we prepare the data for the proxy task, which assumes a fixed-length (L=64) chunk of bootleg features.  To ensure class balance, we randomly sample the same number of chunks from each class.

In [ ]:
def getComposer2IndexMapping(feat_dir):
    composers = []
    for composerDir in sorted(glob.glob('{}/*/'.format(feat_dir))):
        composer = composerDir.split('/')[-2]
        composers.append(composer)
    c_to_i = {c:i for i, c in enumerate(composers)}
    
    return c_to_i, composers

In [ ]:
def getChunkedData_uniformSampling(d, chunkSize, c_to_i):
    '''
    Uniform sampling with 50% overlap.  Doesn't keep page location info.
    '''
    frags = []
    labels = []
    pieceDir2idxRange = {}
    for pieceDir in d:
        merged = np.hstack(d[pieceDir])
        composerIdx = c_to_i[pieceDir.split('/')[-3]]
        startChunkIdx = len(frags)
        for startIdx in range(0, merged.shape[1], chunkSize // 2):
            endIdx = startIdx + chunkSize
            if endIdx <= merged.shape[1]:
                frags.append(merged[:,startIdx:endIdx])
                labels.append(composerIdx)
        endChunkIdx = len(frags)
        pieceDir2idxRange[pieceDir] = (startChunkIdx, endChunkIdx)
    frags = np.array(frags)
    labels = np.array(labels)
    
    return frags, labels, pieceDir2idxRange

In [ ]:
def getChunkedData_balanced(d, chunkSize, samplesPerComposer, composers):
    '''
    Samples the same number of windows from each composer to avoid class imbalance.
    Also keeps the page location for debugging and error analysis.
    '''
    np.random.seed(0)
    
    frags = []
    labels = []
    pagelocs = [] # list of (pieceDir, startPage, endPage) tuples
    
    for composerIdx, composer in enumerate(composers):
        samples, locinfo = sampleFromComposer(d, chunkSize, samplesPerComposer, composer)
        frags.extend(samples)
        labels.extend([composerIdx] * samplesPerComposer)
        pagelocs.extend(locinfo)
        
    frags = np.array(frags)
    labels = np.array(labels)
    
    # shuffle indices
    shuffled_idxs = np.arange(len(labels))
    np.random.shuffle(shuffled_idxs)
    frags = frags[shuffled_idxs]
    labels = labels[shuffled_idxs]
    pagelocs = [pagelocs[i] for i in shuffled_idxs]
    
    return frags, labels, pagelocs

In [ ]:
def sampleFromComposer(d, chunkSize, N, composer):
    '''
    Samples N windows of length chunkSize from the bootleg score data from the specified composer.
    Returns the sampled data, along with piece & page location for error analysis.
    '''
    
    # generate list of all valid sample locations
    allLocs = [] # list of (pieceNum, offset) tuples
    validPieceDirs = [] # pieceDirs that correspond to this composer
    bscores = [] # list of bootleg scores from each piece
    for pieceDir in d:
        if pieceDir.split('/')[-3] == composer:
            pieceNum = len(validPieceDirs)
            validPageFeats = [elem for elem in d[pieceDir] if elem is not None]
            if len(validPageFeats) == 0:
                continue
            merged = np.hstack(validPageFeats)
            for col_offset in range(merged.shape[1] - chunkSize + 1):
                allLocs.append((pieceNum, col_offset))
            validPieceDirs.append(pieceDir)
            bscores.append(merged)

    # generate samples
    frags = []
    pagelocs = []
    sample_idxs = np.random.choice(len(allLocs), N)
    for sample_idx in sample_idxs:
        pieceNum, offset = allLocs[sample_idx]
        pieceDir = validPieceDirs[pieceNum]
        frag = bscores[pieceNum][:, offset:offset+chunkSize]
        locStart = determinePageLocation(d, pieceDir, offset) # (pageLocFloat, pageNum, pageOffset)
        locEnd = determinePageLocation(d, pieceDir, offset + chunkSize - 1)
        frags.append(frag)
        pagelocs.append((pieceDir, locStart, locEnd))

    return frags, pagelocs

In [ ]:
def determinePageLocation(d, pieceDir, offset):
    '''
    Determines the page location of the given bootleg score column offset.  The page location
    is calculated as a floating point number.
    '''
    accum = 0
    pageloc = (-1, -1, -1) # (pageLocFloat, pageNum, pageOffset)
    for pageIdx, elem in enumerate(d[pieceDir]):
        if elem is not None:
            numEvents = elem.shape[1]
            if accum + numEvents > offset:
                frac = (offset - accum) / (numEvents - 1) * 1.0
                pageFloat = pageIdx + frac
                pageloc = (pageFloat, pageIdx, offset - accum) 
                break
            accum += numEvents
    return pageloc

In [ ]:
composer2idx, composers = getComposer2IndexMapping(score_feat_dir)

In [ ]:
chunkSize = 64 # pick a chunk size a bit larger than 64 to allow for additional data augmentation
samplesPerComposer = 3600
X_train, y_train, pageinfo_train = getChunkedData_balanced(d_train, chunkSize, samplesPerComposer, composers)
X_valid, y_valid, pageinfo_valid = getChunkedData_balanced(d_valid, chunkSize, samplesPerComposer//3, composers)
X_test, y_test, pageinfo_test = getChunkedData_balanced(d_test, chunkSize, samplesPerComposer//3, composers)

In [ ]:
X_train.shape, X_valid.shape, X_test.shape

In [ ]:
save_chunks_file = f'{score_feat_dir}/data.chunks{chunkSize}.pkl'
saveToPickle([X_train, y_train, pageinfo_train, X_valid, y_valid, pageinfo_valid, X_test, y_test, pageinfo_test], save_chunks_file)

### Augment Data

In [ ]:
chunkSizes = [64, 128, 256]
shiftss = [[-3,-2,-1,0,1,2,3],[-2,-1,0,1,2],[-1,0,1]]
for shifts in shiftss:
    for chunkSize in chunkSizes:
        score_feat_dir = 'score_feat'
        save_chunks_file = f'{score_feat_dir}/data.chunks{chunkSize}.pkl'
        out_chunks_file = f'{score_feat_dir}/augment/data.chunks{chunkSize}.augment{len(shifts)}.pkl'
        print(out_chunks_file)
        [X_train, y_train, pageinfo_train, X_valid, y_valid, pageinfo_valid, X_test, y_test, pageinfo_test] = loadPickle(save_chunks_file)
        X_train_new = []
        y_train_new = []
        pageinfo_train_new = []
        X_valid_new = []
        y_valid_new = []
        pageinfo_valid_new = []
        X_test_new = []
        y_test_new = []
        pageinfo_test_new = []
        for idx, bscore in enumerate(X_train):
            shift_bscores = editBscoreByShifts(bscore, shifts)
            for shift_bscore in shift_bscores:
                X_train_new.append(shift_bscore)
                y_train_new.append(y_train[idx])
                pageinfo_train_new.append(pageinfo_train[idx])
        for idx, bscore in enumerate(X_valid):
            shift_bscores = editBscoreByShifts(bscore, shifts)
            for shift_bscore in shift_bscores:
                X_valid_new.append(shift_bscore)
                y_valid_new.append(y_valid[idx])
                pageinfo_valid_new.append(pageinfo_valid[idx])
        for idx, bscore in enumerate(X_test):
            shift_bscores = editBscoreByShifts(bscore, shifts)
            for shift_bscore in shift_bscores:
                X_test_new.append(shift_bscore)
                y_test_new.append(y_valid[idx])
                pageinfo_test_new.append(pageinfo_test[idx])

        X_train_new = np.array(X_train_new)
        y_train_new = np.array(y_train_new)
        pageinfo_train_new = np.array(pageinfo_train_new)
        X_valid_new = np.array(X_valid_new)
        y_valid_new = np.array(y_valid_new)
        pageinfo_valid_new = np.array(pageinfo_valid_new)
        X_test_new = np.array(X_test_new)
        y_test_new = np.array(y_test_new)
        pageinfo_test_new = np.array(pageinfo_test_new)

        write_file = [X_train_new, y_train_new, pageinfo_train_new,
                      X_valid_new, y_valid_new, pageinfo_valid_new,
                      X_test_new, y_test_new, pageinfo_test_new]
        with open(out_chunks_file, 'wb') as f:
            pickle.dump(write_file, f, protocol=4)

### Verify data

Compare the bootleg score data chunks against the original png images to verify that the data has been processed correctly.

In [ ]:
chunkSize = 64
score_feat_dir = '../score_feat'
save_chunks_file = f'{score_feat_dir}/augment/data.chunks{chunkSize}.augment7.pkl'
[X_train, y_train, pageinfo_train, X_valid, y_valid, pageinfo_valid, X_test, y_test, pageinfo_test] = loadPickle(save_chunks_file)

In [ ]:
def visualizeBootlegScore(bs, lines = [13, 15, 17, 19, 21, 35, 37, 39, 41, 43]):
    plt.figure(figsize = (10,10))
    plt.imshow(1 - bs, cmap = 'gray', origin = 'lower')
    for l in range(1, bs.shape[0], 2):
        plt.axhline(l, c = 'grey')
    for l in lines:
        plt.axhline(l, c = 'r')

In [ ]:
def getCorrespondingImage(tup, png_dir = 'data/png', showNext = False):
    pieceDir, startLoc, endLoc = tup
    composer = pieceDir.split('/')[-3]
    pieceID = pieceDir.split('/')[-2]
    startpage = startLoc[1]
    if showNext:
        startpage += 1
    pngfile = '{}/{}/{}/{}-{}.png'.format(png_dir, composer, pieceID, pieceID, startpage)
    im = PIL.Image.open(pngfile)
    return im

In [ ]:
i = 5
visualizeBootlegScore(X_train[i])
X_train[i].shape

In [ ]:
pageinfo_train[i]

In [ ]:
im = getCorrespondingImage(pageinfo_train[i], showNext = True)
im

# Prepare data for fastai

In the sections below, we will prepare the data for use with the fastai library.  This is adapted from the fast.ai [ULMFit tutorial](https://github.com/fastai/course-nlp/blob/master/nn-vietnamese.ipynb).

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai import *
from fastai.text import *
import glob

In [ ]:
bs=48

In [ ]:
torch.cuda.set_device(0)

In [ ]:
data_path = Config.data_path()

In [ ]:
name = 'bscore_lm'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

### Target Language Model Databunch

Convert the bootleg score features into string representations of decimal integers.  Generate one document per pdf. Augmentation type is sample pitch shift augmentation.

In [ ]:
path_docs = path/'docs_target-augmented'
path_docs.mkdir(exist_ok=True)

In [ ]:
def convertBinaryToInt(X):
    mask = np.power(2, np.arange(X.shape[0])).reshape((1,-1))
    ints = np.squeeze(mask @ X).astype(np.uint64)
    return list(ints)

In [ ]:
def isValid(matrixList):
    for elem in matrixList:
        if elem is None:
            continue
        else:
            return True
    return False

In [ ]:
def editBscoreByShifts(bscore, shifts):
    # positive = shift right (higher)
    augmented_bscores = []
    for shift in shifts:
        bscore1 = copy.copy(bscore)
        if bscore.shape[0] != 62 or bscore.shape[1] == 0:
            continue
        LH = bscore1[:28,:]
        RH = bscore1[28:,:]
        shift_LH = np.zeros((LH.shape[0]+abs(shift),LH.shape[1]))
        shift_RH = np.zeros((RH.shape[0]+abs(shift),RH.shape[1]))
        if shift < 0:
            shift_LH[:-abs(shift),:] = LH
            shift_RH[:-abs(shift),:] = RH
            new_LH = shift_LH[-LH.shape[0]:,:]
            new_RH = shift_RH[-RH.shape[0]:,:]
        else:
            shift_LH[abs(shift):,:] = LH
            shift_RH[abs(shift):,:] = RH
            new_LH = shift_LH[:LH.shape[0],:]
            new_RH = shift_RH[:RH.shape[0],:]
        bscore1 = np.vstack((new_LH,new_RH))
        augmented_bscores.append(bscore1)
    return augmented_bscores

In [ ]:
def augment_bscore_target(d,shifts):
    output_bscores = [[] for i in range(len(shifts))]
    for page in d:
        if page is None:
            continue
        bscores = editBscoreByShifts(page, shifts)
        for idx, new_page in enumerate(bscores):
            output_bscores[idx].append(new_page)
    return output_bscores

In [ ]:
def generateBootlegStringFiles(bscore_feats_file, outdir):
    '''
    Converts the bootleg score features to string decimal representation, and writes them
    to text files in the specified directory.
    '''
    
    with open(bscore_feats_file, 'rb') as f:
        bscore_feats = pickle.load(f)[0]
                
    for pieceDir in bscore_feats: # e.g. score_feat/Bach/00748/
        pid = pieceDir.split('/')[-2] # 00748
        if isValid(bscore_feats[pieceDir]): # has at least one valid page of features
            shifts = [-3,-2,-1,0,1,2,3]
            bscores = augment_bscore_target(bscore_feats[pieceDir], shifts)
            for idx, shift in enumerate(shifts):
                outfile = outdir/ (pid +f'({shift}).txt')
                print(outfile)
                
                with open(outfile,'w') as fout:
                    for binaryMatrix in bscores[idx]:
                        if binaryMatrix is not None:
                            ints = convertBinaryToInt(binaryMatrix)
                            pageStr = ' '.join([str(i) for i in ints])
                            fout.write(pageStr)
                            fout.write('\n\n')
                    fout.write('</doc>')

In [ ]:
def bscore2textfile(infile, outfile, min_thresh = 100):
    '''
    Converts a bootleg score .pkl file to text and writes to the specified output file.
    '''
    with open(infile, 'rb') as f:
        d = pickle.load(f)
    
    
    for idx,shift in enumerate(shifts):
        new_ext = os.path.splitext(os.path.basename(outfile))[0]+'({})'.format(shift)+os.path.splitext(os.path.basename(outfile))[1]
        outfile_shift ='{}/{}'.format(os.path.dirname(outfile), new_ext)
        with open(outfile_shift, 'w') as fout:
            d = bscores[idx]
            for l in d: # each page
                if len(l) > min_thresh: # to avoid filler pages
                    pageStr = ' '.join([str(i) for i in l])
                    fout.write(pageStr)
                    fout.write('\n\n')
            fout.write('</doc>')

In [ ]:
generateBootlegStringFiles(save_pages_file, path_docs)

In [ ]:
path_docs.ls()[:5]

In [ ]:
basicTokenizer = Tokenizer(pre_rules=[], post_rules=[])
lm_target_data = (TextList.from_folder(path_docs, processor=[OpenFileProcessor(), TokenizeProcessor(tokenizer=basicTokenizer), NumericalizeProcessor()])
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

lm_target_data.save(path/'lm_target_databunch-augmented')

### IMSLP Language Model Databunch

Same as above, but using the entire IMSLP piano bootleg score dataset.

In [ ]:
!git clone https://github.com/HMC-MIR/piano_bootleg_scores.git

In [ ]:
imslp_bscores_filelist = 'imslp_bscores.list'
!find piano_bootleg_scores/imslp_bootleg_dir-v1/ -name *.pkl > {imslp_bscores_filelist}

In [ ]:
path_docs = path/'docs_imslp-augmented'
path_docs.mkdir(exist_ok=True)

In [ ]:
def decodeColumn(num):
    col = []
    for i in range(62):
        col.insert(0,num%2)
        num = int(num/2)
    return col

In [ ]:
def bootlegHash(arr):
    bitstring = ""
    for i in range(len(arr)):
        if arr[i]==1:
            bitstring+="1"
        else:
            bitstring +="0"
    bitstring = bitstring+"00"
    hashint = int(bitstring, 2)
    hashint = np.uint64(hashint)
    return hashint

In [ ]:
def augment_bscore(d,shifts):
    output_bscores = [[] for i in range(len(shifts))]
    
    for page in d:
        page_bscore = []
        for column in page:
            col = decodeColumn(column)
            page_bscore.append(col)
        page_bscore = np.array(page_bscore).T
        bscores = editBscoreByShifts(page_bscore, shifts)
        for idx, bscore in enumerate(bscores):
            hashed_bscore = []
            for col in bscore.T:
                hashint = bootlegHash(col)
                hashed_bscore.append(hashint)
            output_bscores[idx].append(hashed_bscore)
    return output_bscores

In [ ]:
def editBscoreByShifts(bscore, shifts):
    # positive = shift right (higher)
    augmented_bscores = []
    for shift in shifts:
        bscore1 = copy.copy(bscore)
        if bscore.shape[0] != 62 or bscore.shape[1] == 0:
            continue
        LH = bscore1[:28,:]
        RH = bscore1[28:,:]
        shift_LH = np.zeros((LH.shape[0]+abs(shift),LH.shape[1]))
        shift_RH = np.zeros((RH.shape[0]+abs(shift),RH.shape[1]))
        if shift < 0:
            shift_LH[:-abs(shift),:] = LH
            shift_RH[:-abs(shift),:] = RH
            new_LH = shift_LH[-LH.shape[0]:,:]
            new_RH = shift_RH[-RH.shape[0]:,:]
        else:
            shift_LH[abs(shift):,:] = LH
            shift_RH[abs(shift):,:] = RH
            new_LH = shift_LH[:LH.shape[0],:]
            new_RH = shift_RH[:RH.shape[0],:]
        bscore1 = np.vstack((new_LH,new_RH))
        augmented_bscores.append(bscore1)
    return augmented_bscores

In [ ]:
def bscore2textfile(infile, outfile, min_thresh = 100):
    '''
    Converts a bootleg score .pkl file to text and writes to the specified output file.
    '''
    with open(infile, 'rb') as f:
        d = pickle.load(f)
    shifts = [-3,-2,-1,0,1,2,3]
    bscores = augment_bscore(d, shifts)
    
    for idx,shift in enumerate(shifts):
        new_ext = os.path.splitext(os.path.basename(outfile))[0]+'({})'.format(shift)+os.path.splitext(os.path.basename(outfile))[1]
        outfile_shift ='{}/{}'.format(os.path.dirname(outfile), new_ext)
        with open(outfile_shift, 'w') as fout:
            d = bscores[idx]
            for l in d: # each page
                if len(l) > min_thresh: # to avoid filler pages
                    pageStr = ' '.join([str(i) for i in l])
                    fout.write(pageStr)
                    fout.write('\n\n')
            fout.write('</doc>')

In [ ]:
def imslp2text(filelist, outdir):
    with open(filelist, 'r') as f:
        for line in f:
            bscorefile = line.strip() # path/to/dest/283513.pkl
            fileid = os.path.splitext(os.path.basename(bscorefile))[0] # e.g. 283513
            outfile = outdir/f'{fileid}.txt'
            bscore2textfile(bscorefile, outfile)

In [ ]:
imslp2text(imslp_bscores_filelist, path_docs)

In [ ]:
path_docs.ls()[:5]

In [ ]:
basicTokenizer = Tokenizer(pre_rules=[], post_rules=[])
lm_imslp_data = (TextList.from_folder(path_docs, processor=[OpenFileProcessor(), TokenizeProcessor(tokenizer=basicTokenizer), NumericalizeProcessor()])
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

lm_imslp_data.save(path/'lm_imslp_databunch-augmented')

### Classification data

Here we prepare the train.csv, valid.csv, and test.csv files for the proxy classification task.

In [ ]:
def generateBootlegCSVFiles(bscore_feats_file, idx2composer, outfile_train, outfile_valid, outfile_test):
    '''
    Generates train.csv and test.csv from bootleg score fragments.
    '''
    with open(bscore_feats_file, 'rb') as f:
        X_train, y_train, _, X_valid, y_valid, _, X_test, y_test, _ = pickle.load(f)
        
    y_train = [idx2composer[idx] for idx in y_train]
    y_valid = [idx2composer[idx] for idx in y_valid]
    y_test = [idx2composer[idx] for idx in y_test]
    
    with open(outfile_train, 'w') as fout:
        fout.write('label,text\n')
        for i in range(X_train.shape[0]):
            ints = convertBinaryToInt(X_train[i,:,:])
            pageStr = ' '.join([str(i) for i in ints])
            fout.write(f'{y_train[i]},')
            fout.write(pageStr)
            fout.write('\n')

    with open(outfile_valid, 'w') as fout:
        fout.write('label,text\n')
        for i in range(X_valid.shape[0]):
            ints = convertBinaryToInt(X_valid[i,:,:])
            pageStr = ' '.join([str(i) for i in ints])
            fout.write(f'{y_valid[i]},')
            fout.write(pageStr)
            fout.write('\n')

    with open(outfile_test, 'w') as fout:
        fout.write('label,text\n')
        for i in range(X_test.shape[0]):
            ints = convertBinaryToInt(X_test[i,:,:])
            pageStr = ' '.join([str(i) for i in ints])
            fout.write(f'{y_test[i]},')
            fout.write(pageStr)
            fout.write('\n')
            
    return

In [ ]:
save_chunks_file = 'score_feat/augment/data.chunks256.augment3.pkl'

In [ ]:
csv_train_file = path/'augment/train256.augment3.csv'
csv_valid_file = path/'augment/valid256.augment3.csv'
csv_test_file = path/'augment/test256.augment3.csv'
generateBootlegCSVFiles(save_chunks_file, composers, csv_train_file, csv_valid_file, csv_test_file)

We also generate csv files for evaluating on the original page classification task.

In [ ]:
def generateFullPageCSVFiles(bscore_pages_file, outfile_train, outfile_valid, outfile_test):
    '''
    Generates csv files for the original task of classifying full pages of music.
    '''
    
    with open(bscore_pages_file, 'rb') as f:
        d, pieceDirs_train, pieceDirs_valid, pieceDirs_test = pickle.load(f)
    
    generateFullPageCSV(d, pieceDirs_train, outfile_train)
    generateFullPageCSV(d, pieceDirs_valid, outfile_valid)
    generateFullPageCSV(d, pieceDirs_test, outfile_test)

In [ ]:
def generateFullPageCSV(d, keys, outfile):
    
    with open(outfile, 'w') as fout:
        fout.write('label,text\n')
        for pieceDir in keys: # e.g. score_feat/Bach/00748/
            composer = pieceDir.split('/')[-3]
            for m in d[pieceDir]: # d[pieceDir] -> list of binary bootleg score matrices, one per page
                if m is not None:
                    ints = convertBinaryToInt(m)
                    textStr = ' '.join([str(i) for i in ints])
                    fout.write(f'{composer},{textStr}\n')

In [ ]:
csv_train_file = path/'train.fullpage-augment7.csv'
csv_valid_file = path/'valid.fullpage-augment7.csv'
csv_test_file = path/'test.fullpage-augment7.csv'
save_pages_file='score_feat/data.pages-augment7.pkl'
generateFullPageCSVFiles(save_pages_file, csv_train_file, csv_valid_file, csv_test_file)

Finally, we also generate csv files to facilitate evaluating fixed-length classifiers on the full page classification task.  These classifiers will be applied to multiple windows of features, and the predictions will be averaged.

In [ ]:
def generateEnsembleCSV(bscore_pages_file, chunkSz, outfile_test):
    '''
    Generates a csv file to facilitate evaluating fixed-length classifiers on the full page classification task.
    Each line in the file corresponds to a fixed-length window of samples within a page.  The predictions from
    all windows within a single page can then be averaged and evaluated.
    '''
    with open(bscore_pages_file, 'rb') as f:
        d, pieceDirs_train, pieceDirs_valid, pieceDirs_test = pickle.load(f)
        
    with open(outfile_test, 'w') as fout:
        fout.write('id,label,text\n')
        for pieceDir in pieceDirs_test: # e.g. score_feat/Bach/00748/
            pieceID = pieceDir.split('/')[-2]
            composer = pieceDir.split('/')[-3]
            for i, m in enumerate(d[pieceDir]): # d[pieceDir] -> list of binary bootleg score matrices, one per page
                if m is not None and m.shape[1] > 0:
                    if m.shape[1] <= chunkSz: # only 1 window
                        ints = convertBinaryToInt(m)
                        textStr = ' '.join([str(i) for i in ints])
                        idString = f'{pieceID}_{i}_0' # id: pieceID_pageIdx_chunkIdx
                        fout.write(f'{idString},{composer},{textStr}\n')
                    else: # multiple windows
                        numWindows = int(np.ceil(m.shape[1]/(chunkSz/2))) - 1 # hop by half the chunk size
                        for j in range(numWindows - 1):
                            startIdx = chunkSz // 2 * j
                            endIdx = startIdx + chunkSz
                            ints = convertBinaryToInt(m[:,startIdx:endIdx])
                            textStr = ' '.join([str(i) for i in ints])
                            idString = f'{pieceID}_{i}_{j}' # id: pieceID_pageIdx_chunkIdx
                            fout.write(f'{idString},{composer},{textStr}\n')
                        # handle last window
                        ints = convertBinaryToInt(m[:,-chunkSz:])
                        textStr = ' '.join([str(i) for i in ints])
                        idString = f'{pieceID}_{i}_{numWindows-1}' 
                        fout.write(f'{idString},{composer},{textStr}\n')

In [ ]:
csv_test_file = path/'test.ensemble256-auugment7.csv'
generateEnsembleCSV(save_pages_file,256, csv_test_file)

# Data Preparation for Transformer models

Here we prepare the data for training and testing the Transformer-based models.  Instead of using decimal string representations, we represent each 62-bit bootleg score feature as a sequence of 8 one-byte characters.  Rather than generating these from scratch, we will simply convert the existing files to the new format.

### Prep data for language modeling

In [ ]:
def generateLMTrainFiles(indir, out_train, out_valid, val_frac=0.1):
    
    # split train/validation by file
    filelist = sorted(glob.glob('{}/*.txt'.format(indir)))
    np.random.seed(0)
    np.random.shuffle(filelist)
    endIdx = int(len(filelist) * (1-val_frac)) + 1
    train_files = filelist[0:endIdx]
    valid_files = filelist[endIdx:]
    
    # convert to binary string representation
    convertToByteChars(train_files, out_train)
    convertToByteChars(valid_files, out_valid)

In [ ]:
def convertToByteChars(filelist, outfile):
    '''
    Split each 62-bit bootleg score feature into 8 bytes, and express each byte as a single character.
    Consecutive bootleg score feature `words' will be separated by space.
    '''
    with open(outfile, 'w') as fout:
        for infile in filelist:
            with open(infile, 'r') as fin:
                for line in fin:
                    line = line.strip()
                    if len(line) > 0:
                        if line == '</doc>':
                            pass # skip
                        else:
                            converted = convertLineToCharSeq(line)
                            fout.write(f'{converted}\n')
            fout.write('\n')

In [ ]:
def convertLineToCharSeq(line):
    ints = [int(p) for p in line.split()]
    result = ' '.join([int2charseq(i) for i in ints])
    return result

In [ ]:
def int2charseq(int64):
    chars = ''
    for i in range(8):
        numshift = i * 8
        charidx = (int64 >> numshift) & 255
        chars += chr(19968 + charidx) # 19968 ensures that all chars are chinese characters (not newline, space, etc)
    return ''.join(chars)

In [ ]:
bpe_path = path/'bpe_data'
bpe_path.mkdir(exist_ok=True, parents=True)

In [ ]:
# Convert target data
lm_train_file = bpe_path/'bpe_lm_target_train-augmented.txt'
lm_valid_file = bpe_path/'bpe_lm_target_valid-augmented.txt'
dir_to_convert = path/'docs_target-augmented'
generateLMTrainFiles(dir_to_convert, lm_train_file, lm_valid_file)

In [ ]:
# Convert IMSLP data
lm_train_file = bpe_path/'bpe_lm_imslp_train-augmented.txt'
lm_valid_file = bpe_path/'bpe_lm_imslp_valid-augmented.txt'
dir_to_convert = path/'docs_imslp-augmented'
generateLMTrainFiles(dir_to_convert, lm_train_file, lm_valid_file)

### Prep data for classification

In [ ]:
def convertSingleCSVFile(infile, outfile):
    '''
    Convert .csv file with decimal string representation of bootleg score features to
    a .csv file with byte character representation.
    '''
    with open(infile, 'r') as f:
        lines = f.readlines()
    with open(outfile, 'w') as fout:
        for i, line in enumerate(lines):
            if i==0: 
                fout.write(line) # header
            else:
                parts = line.strip().split(',')
                feats = parts.pop()
                charseq = convertLineToCharSeq(feats)
                strToWrite = ','.join(parts) + ',' + charseq + '\n'
                fout.write(strToWrite)

In [ ]:
def convertAllCSVFiles(indir, outdir):
    assert indir != outdir
    os.makedirs(outdir, exist_ok = True)
    for infile in glob.glob(f'{indir}/*.csv'):
        print(f'Converting {os.path.basename(infile)}')
        basename = os.path.splitext(os.path.basename(infile))[0]
        outfile = f'{outdir}/{basename}.char.csv'
        convertSingleCSVFile(infile, outfile)

In [ ]:
convertAllCSVFiles(str(path), str(bpe_path))

In [ ]:
convertAllCSVFiles(str(path)+'/augment', str(bpe_path))